## dataset

In [1]:
from trl.trainer import ConstantLengthDataset

[2024-03-17 14:54:24,055] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


- ConstantLengthDataset
    - seq_length=args.seq_length,

## `prepare_model_for_kbit_training`

This method wraps the entire protocol for preparing a model before running a training. 
This includes:
- 1- Cast the layernorm in fp32 
- 2- making output embedding layer require grads 
- 3- Add the upcasting of the lm head to fp32

```
if not is_gptq_quantized:
    # cast all non INT8 parameters to fp32
    for param in model.parameters():
        if (param.dtype == torch.float16) or (param.dtype == torch.bfloat16):
            param.data = param.data.to(torch.float32)
```

## peft_module_casting_to_bf16

```
def peft_module_casting_to_bf16(model):
    from peft.tuners.tuners_utils import BaseTunerLayer

    for name, module in model.named_modules():
        if isinstance(module, BaseTunerLayer):
            module = module.to(torch.bfloat16)
        elif isinstance(module, torch.nn.LayerNorm) or "norm" in name:
            module = module.to(torch.float32)
        elif any(x in name for x in ["lm_head", "embed_tokens", "wte", "wpe"]):
            if hasattr(module, "weight"):
                if module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)
```